Policy中，每個險種展開，  
各險種的 保額123 + 保費 + 全部險種保費總額

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import numpy as np
import datetime

from sklearn import cluster, datasets

import time
from IPython.display import clear_output

%matplotlib inline
plt.rcParams['font.family'] = ['SimHei'] #  set font for chinese in matplotlib figure

In [23]:
df_policy_raw = pd.read_csv('../Tbrain_Insurance/policy_claim/policy_0702.csv')
df_claim_raw = pd.read_csv('../Tbrain_Insurance/policy_claim/claim_0702.csv')
df_training_raw = pd.read_csv('../Tbrain_Insurance/training-set.csv')
df_testing_raw = pd.read_csv('../Tbrain_Insurance/testing-set.csv')
df_policy_coverage = pd.read_csv('../Tbrain_Insurance/df_policy_coverage-peter3.csv')

In [3]:
df_policy = df_policy_raw 
df_claim = df_claim_raw 
df_training = df_training_raw
df_testing = df_testing_raw
pd.set_option('display.max_columns', None)

In [4]:
#df_policy_coverage = df_policy_coverage.merge(df_policy, how='left', on ='Policy_Number')

In [31]:
print(df_policy_coverage.shape)
df_policy_coverage.head(5)

(351273, 242)


,Policy_Number,total_Premium,00I_Insured_Amount1,00I_Insured_Amount2,00I_Insured_Amount3,00I_Premium,01A_Insured_Amount1,01A_Insured_Amount2,01A_Insured_Amount3,01A_Premium,01J_Insured_Amount1,01J_Insured_Amount2,01J_Insured_Amount3,01J_Premium,02K_Insured_Amount1,02K_Insured_Amount2,02K_Insured_Amount3,02K_Premium,03L_Insured_Amount1,03L_Insured_Amount2,03L_Insured_Amount3,03L_Premium,04M_Insured_Amount1,04M_Insured_Amount2,04M_Insured_Amount3,04M_Premium,05E_Insured_Amount1,05E_Insured_Amount2,05E_Insured_Amount3,05E_Premium,05N_Insured_Amount1,05N_Insured_Amount2,05N_Insured_Amount3,05N_Premium,06F_Insured_Amount1,06F_Insured_Amount2,06F_Insured_Amount3,06F_Premium,07P_Insured_Amount1,07P_Insured_Amount2,07P_Insured_Amount3,07P_Premium,08H_Insured_Amount1,08H_Insured_Amount2,08H_Insured_Amount3,08H_Premium,09@_Insured_Amount1,09@_Insured_Amount2,09@_Insured_Amount3,09@_Premium,09I_Insured_Amount1,09I_Insured_Amount2,09I_Insured_Amount3,09I_Premium,10A_Insured_Amount1,10A_Insured_Amount2,10A_Insured_Amount3,10A_Premium,12L_Insured_Amount1,12L_Insured_Amount2,12L_Insured_Amount3,12L_Premium,14E_Insured_Amount1,14E_Insured_Amount2,14E_Insured_Amount3,14E_Premium,14N_Insured_Amount1,14N_Insured_Amount2,14N_Insured_Amount3,14N_Premium,15F_Insured_Amount1,15F_Insured_Amount2,15F_Insured_Amount3,15F_Premium,15O_Insured_Amount1,15O_Insured_Amount2,15O_Insured_Amount3,15O_Premium,16G_Insured_Amount1,16G_Insured_Amount2,16G_Insured_Amount3,16G_Premium,16P_Insured_Amount1,16P_Insured_Amount2,16P_Insured_Amount3,16P_Premium,18@_Insured_Amount1,18@_Insured_Amount2,18@_Insured_Amount3,18@_Premium,18I_Insured_Amount1,18I_Insured_Amount2,18I_Insured_Amount3,18I_Premium,20B_Insured_Amount1,20B_Insured_Amount2,20B_Insured_Amount3,20B_Premium,20K_Insured_Amount1,20K_Insured_Amount2,20K_Insured_Amount3,20K_Premium,25G_Insured_Amount1,25G_Insured_Amount2,25G_Insured_Amount3,25G_Premium,26H_Insured_Amount1,26H_Insured_Amount2,26H_Insured_Amount3,26H_Premium,27I_Insured_Amount1,27I_Insured_Amount2,27I_Insured_Amount3,27I_Premium,29B_Insured_Amount1,29B_Insured_Amount2,29B_Insured_Amount3,29B_Premium,29K_Insured_Amount1,29K_Insured_Amount2,29K_Insured_Amount3,29K_Premium,32N_Insured_Amount1,32N_Insured_Amount2,32N_Insured_Amount3,32N_Premium,33F_Insured_Amount1,33F_Insured_Amount2,33F_Insured_Amount3,33F_Premium,33O_Insured_Amount1,33O_Insured_Amount2,33O_Insured_Amount3,33O_Premium,34P_Insured_Amount1,34P_Insured_Amount2,34P_Insured_Amount3,34P_Premium,35H_Insured_Amount1,35H_Insured_Amount2,35H_Insured_Amount3,35H_Premium,36I_Insured_Amount1,36I_Insured_Amount2,36I_Insured_Amount3,36I_Premium,37J_Insured_Amount1,37J_Insured_Amount2,37J_Insured_Amount3,37J_Premium,40M_Insured_Amount1,40M_Insured_Amount2,40M_Insured_Amount3,40M_Premium,41E_Insured_Amount1,41E_Insured_Amount2,41E_Insured_Amount3,41E_Premium,41N_Insured_Amount1,41N_Insured_Amount2,41N_Insured_Amount3,41N_Premium,42F_Insured_Amount1,42F_Insured_Amount2,42F_Insured_Amount3,42F_Premium,45@_Insured_Amount1,45@_Insured_Amount2,45@_Insured_Amount3,45@_Premium,46A_Insured_Amount1,46A_Insured_Amount2,46A_Insured_Amount3,46A_Premium,47B_Insured_Amount1,47B_Insured_Amount2,47B_Insured_Amount3,47B_Premium,51O_Insured_Amount1,51O_Insured_Amount2,51O_Insured_Amount3,51O_Premium,55J_Insured_Amount1,55J_Insured_Amount2,55J_Insured_Amount3,55J_Premium,56B_Insured_Amount1,56B_Insured_Amount2,56B_Insured_Amount3,56B_Premium,56K_Insured_Amount1,56K_Insured_Amount2,56K_Insured_Amount3,56K_Premium,57C_Insured_Amount1,57C_Insured_Amount2,57C_Insured_Amount3,57C_Premium,57L_Insured_Amount1,57L_Insured_Amount2,57L_Insured_Amount3,57L_Premium,65K_Insured_Amount1,65K_Insured_Amount2,65K_Insured_Amount3,65K_Premium,66C_Insured_Amount1,66C_Insured_Amount2,66C_Insured_Amount3,66C_Premium,66L_Insured_Amount1,66L_Insured_Amount2,66L_Insured_Amount3,66L_Premium,67D_Insured_Amount1,67D_Insured_Amount2,67D_Insured_Amount3,67D_Premium,68E_Insured_Amount1,68E_Insured_Amount2,68E_Insured_Amount3,68E_Premium,68N_Insured

In [37]:
#df.filter(regex=("d.*"))
coverage_premium = df_policy_coverage.filter(regex=('.*_Premium'))
coverage_premium['Policy_Number'] = df_policy_coverage['Policy_Number']
print(coverage_premium.shape)
coverage_premium.head(5)

(351273, 62)


d:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,total_Premium,00I_Premium,01A_Premium,01J_Premium,02K_Premium,03L_Premium,04M_Premium,05E_Premium,05N_Premium,06F_Premium,07P_Premium,08H_Premium,09@_Premium,09I_Premium,10A_Premium,12L_Premium,14E_Premium,14N_Premium,15F_Premium,15O_Premium,16G_Premium,16P_Premium,18@_Premium,18I_Premium,20B_Premium,20K_Premium,25G_Premium,26H_Premium,27I_Premium,29B_Premium,29K_Premium,32N_Premium,33F_Premium,33O_Premium,34P_Premium,35H_Premium,36I_Premium,37J_Premium,40M_Premium,41E_Premium,41N_Premium,42F_Premium,45@_Premium,46A_Premium,47B_Premium,51O_Premium,55J_Premium,56B_Premium,56K_Premium,57C_Premium,57L_Premium,65K_Premium,66C_Premium,66L_Premium,67D_Premium,68E_Premium,68N_Premium,70G_Premium,70P_Premium,71H_Premium,72@_Premium,Policy_Number
0,2714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1551.0,1163.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79110176bf64b5094c19aad785aeac56e36cb609
1,3129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2031.0,1098.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61a449f6180b23379a584af904d7412205d8ff8c
2,3284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,149.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1289.0,1846.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1a1f5f582bf1ff23467352e8f2fd31acefddc654
3,183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,183.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6b420a887376f4dce53e22e600f2ca56aadec0ee
4,7529,0.0,184.0,0.0,0.0,0.0,0.0,0.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,95.0,0.0,0.0,0.0,1879.0,664.0,658.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,408.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3406.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15a5e6b7c385d23c01bf4eaa49f310d638d7d3d9


In [50]:
fsex = df_policy[['Policy_Number','fsex']]
fsex['fsex'] = fsex['fsex'].fillna(value=' ')
fsex = pd.get_dummies(fsex, columns = ['fsex'])
fsex = fsex.groupby(fsex['Policy_Number'])[fsex.columns[1:]].first().reset_index()

d:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [52]:
print(fsex['fsex_ '].value_counts())
print(fsex['fsex_1'].value_counts())
print(fsex['fsex_2'].value_counts())
fsex.head(5)

0    294958
1     56315
Name: fsex_ , dtype: int64
0    209567
1    141706
Name: fsex_1, dtype: int64
0    198021
1    153252
Name: fsex_2, dtype: int64


,Policy_Number,fsex_,fsex_1,fsex_2
0,000061141b237e8619efedcd6939fddeff05b9a5,0,0,1
1,0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5,0,1,0
2,0001221abee6f59d6d40eeb1ba9ffa0f4f92b36b,0,1,0
3,000148bbf40f28bfc860c0d272aadc429e19e2d1,0,1,0
4,0001ad0ad0618f6d2e2c1875105adce6bcd77c2c,0,0,1


In [53]:
coverage_premium_fsex = coverage_premium.merge(fsex, how='left', on ='Policy_Number')

In [54]:
print(coverage_premium_fsex.shape)
coverage_premium_fsex.head(5)

(351273, 65)


,total_Premium,00I_Premium,01A_Premium,01J_Premium,02K_Premium,03L_Premium,04M_Premium,05E_Premium,05N_Premium,06F_Premium,07P_Premium,08H_Premium,09@_Premium,09I_Premium,10A_Premium,12L_Premium,14E_Premium,14N_Premium,15F_Premium,15O_Premium,16G_Premium,16P_Premium,18@_Premium,18I_Premium,20B_Premium,20K_Premium,25G_Premium,26H_Premium,27I_Premium,29B_Premium,29K_Premium,32N_Premium,33F_Premium,33O_Premium,34P_Premium,35H_Premium,36I_Premium,37J_Premium,40M_Premium,41E_Premium,41N_Premium,42F_Premium,45@_Premium,46A_Premium,47B_Premium,51O_Premium,55J_Premium,56B_Premium,56K_Premium,57C_Premium,57L_Premium,65K_Premium,66C_Premium,66L_Premium,67D_Premium,68E_Premium,68N_Premium,70G_Premium,70P_Premium,71H_Premium,72@_Premium,Policy_Number,fsex_,fsex_1,fsex_2
0,2714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1551.0,1163.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79110176bf64b5094c19aad785aeac56e36cb609,0,0,1
1,3129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2031.0,1098.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61a449f6180b23379a584af904d7412205d8ff8c,0,0,1
2,3284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,149.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1289.0,1846.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1a1f5f582bf1ff23467352e8f2fd31acefddc654,0,0,1
3,183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,183.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6b420a887376f4dce53e22e600f2ca56aadec0ee,0,1,0
4,7529,0.0,184.0,0.0,0.0,0.0,0.0,0.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,95.0,0.0,0.0,0.0,1879.0,664.0,658.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,408.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3406.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15a5e6b7c385d23c01bf4eaa49f310d638d7d3d9,1,0,0


In [196]:
coverage_premium_Juridical = coverage_premium_fsex[coverage_premium_fsex['fsex_ ']==1]
kmeans_fit_coverage_premium_Juridical = cluster.KMeans(n_clusters = 25).fit(coverage_premium_Juridical.iloc[:,0:-4]) #*****調整群數改這裡

kmeans_coverage_premium_labels = kmeans_fit_coverage_premium_Juridical.labels_
print("分群結果：")
print(kmeans_coverage_premium_labels)
print("---")

coverage_premium_Juridical['kmeans_coverage_premium_Juridical'] = kmeans_coverage_premium_labels
coverage_premium_Juridical['kmeans_coverage_premium_Juridical'].value_counts()

分群結果：
[ 3 24 24 ... 17  5 23]
---


d:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


24    20956
3     15758
17     7149
0      3067
7      2169
15     1518
2       992
23      910
19      744
13      564
10      483
20      470
5       374
18      344
8       233
1       197
22      137
21      134
6        52
16       37
14       13
4         6
9         4
12        3
11        1
Name: kmeans_coverage_premium_Juridical, dtype: int64

In [197]:
coverage_premium_Juridical[coverage_premium_Juridical['kmeans_coverage_premium_Juridical']==8].describe()

,total_Premium,00I_Premium,01A_Premium,01J_Premium,02K_Premium,03L_Premium,04M_Premium,05E_Premium,05N_Premium,06F_Premium,07P_Premium,08H_Premium,09@_Premium,09I_Premium,10A_Premium,12L_Premium,14E_Premium,14N_Premium,15F_Premium,15O_Premium,16G_Premium,16P_Premium,18@_Premium,18I_Premium,20B_Premium,20K_Premium,25G_Premium,26H_Premium,27I_Premium,29B_Premium,29K_Premium,32N_Premium,33F_Premium,33O_Premium,34P_Premium,35H_Premium,36I_Premium,37J_Premium,40M_Premium,41E_Premium,41N_Premium,42F_Premium,45@_Premium,46A_Premium,47B_Premium,51O_Premium,55J_Premium,56B_Premium,56K_Premium,57C_Premium,57L_Premium,65K_Premium,66C_Premium,66L_Premium,67D_Premium,68E_Premium,68N_Premium,70G_Premium,70P_Premium,71H_Premium,72@_Premium,fsex_,fsex_1,fsex_2,kmeans_coverage_premium_Juridical
count,233.000000,233.0,233.000000,233.0,233.0,233.000000,233.0,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000,233.0,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000,233.0,233.000000,233.00000,233.0,233.000000,233.0,233.000000,233.000000,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.000000,233.0,233.0,233.0,233.000000,233.000000,233.000000,233.0,233.000000,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0
mean,72822.201717,0.0,302.519313,0.0,0.0,85.145923,0.0,656.905579,7342.261803,324.969957,10.360515,23.351931,592.506438,0.0,73.424893,35.896996,40.120172,962.506438,207.819742,110.381974,2598.103004,1845.969957,36.793991,0.0,35.596567,11.55794,0.0,6.527897,0.0,1005.673820,586.360515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.995708,0.0,0.0,0.0,55850.523605,35.845494,13.072961,0.0,5.008584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
std,8288.325364,0.0,1485.285863,0.0,0.0,238.933289,0.0,363.388174,5495.861497,1083.734016,78.947814,110.945566,829.428512,0.0,116.086452,80.087386,137.083234,2181.124214,468.135638,533.848838,984.385717,609.696210,66.467782,0.0,94.614601,12.44392,0.0,37.251132,0.0,295.614156,297.522194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351.014251,0.0,0.0,0.0,7919.532900,386.286099,199.550095,0.0,19.294003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,59407.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,34947.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
25%,66156.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,649.000000,3160.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2000.000000,1512.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,887.000000,484.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,50445.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
50%,71133.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,792.000000,6218.000000,0.000000,0.000000,0.000000,166.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2585.000000,1837.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,1082.000000,591.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,54778.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
75%,78645.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,896.000000,10956.000000,0.000000,0.000000,0.000000,974.000000,0.0,164.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3182.000000,2105.000000,78.000000,0.0,0.000000,26.00000,0.0,0.000000,0.0,1082.000000,591.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,61704.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
max,96652.000000,0.0,11863.000000,0.0,0.0,1060.000000,0.0,1188.000

In [198]:
coverage_premium_Juridical[coverage_premium_Juridical['kmeans_coverage_premium_Juridical']==1].describe()

,total_Premium,00I_Premium,01A_Premium,01J_Premium,02K_Premium,03L_Premium,04M_Premium,05E_Premium,05N_Premium,06F_Premium,07P_Premium,08H_Premium,09@_Premium,09I_Premium,10A_Premium,12L_Premium,14E_Premium,14N_Premium,15F_Premium,15O_Premium,16G_Premium,16P_Premium,18@_Premium,18I_Premium,20B_Premium,20K_Premium,25G_Premium,26H_Premium,27I_Premium,29B_Premium,29K_Premium,32N_Premium,33F_Premium,33O_Premium,34P_Premium,35H_Premium,36I_Premium,37J_Premium,40M_Premium,41E_Premium,41N_Premium,42F_Premium,45@_Premium,46A_Premium,47B_Premium,51O_Premium,55J_Premium,56B_Premium,56K_Premium,57C_Premium,57L_Premium,65K_Premium,66C_Premium,66L_Premium,67D_Premium,68E_Premium,68N_Premium,70G_Premium,70P_Premium,71H_Premium,72@_Premium,fsex_,fsex_1,fsex_2,kmeans_coverage_premium_Juridical
count,197.000000,197.000000,197.000000,197.000000,197.0,197.000000,197.0,197.000000,197.000000,197.000000,197.0,197.000000,197.000000,197.0,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000,197.0,197.000000,197.000000,197.0,197.000000,197.0,197.000000,197.000000,197.000000,197.0,197.0,197.000000,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.000000,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.0,197.0
mean,69805.126904,54345.629442,971.131980,14.223350,0.0,76.796954,0.0,634.233503,5435.157360,445.152284,0.0,15.944162,406.106599,0.0,55.934010,49.076142,10.482234,752.050761,37.076142,570.157360,2422.791878,1711.532995,42.690355,0.0,2.903553,6.654822,0.0,10.101523,0.0,1022.619289,603.147208,102.532995,0.0,0.0,56.903553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.096447,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
std,10535.972370,8866.919204,2669.706411,199.634234,0.0,226.426091,0.0,364.595479,4305.088468,1258.459391,0.0,90.636817,642.303932,0.0,108.925179,96.859351,73.418190,2015.050300,211.074364,1061.253394,891.017306,514.575816,68.422163,0.0,28.743337,10.670512,0.0,44.016924,0.0,299.844147,336.453690,865.703103,0.0,0.0,563.309106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.891901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,53809.000000,30602.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
25%,61249.000000,47345.000000,0.000000,0.000000,0.0,0.000000,0.0,620.000000,2320.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1924.000000,1412.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,887.000000,484.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
50%,67490.000000,53234.000000,0.000000,0.000000,0.0,0.000000,0.0,750.000000,4844.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2389.000000,1722.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,992.000000,484.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
75%,78464.000000,60705.000000,0.000000,0.000000,0.0,0.000000,0.0,894.000000,7395.000000,0.000000,0.0,0.000000,729.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2784.000000,1957.000000,82.000000,0.0,0.000000,21.000000,0.0,0.000000,0.0,1170.000000,591.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
max,96301.000000,79570.000000,14770.000000,2802.000000,0.0,944.000000,0.0,1124.000000,23735.000000,7208

In [199]:
coverage_premium_Juridical[coverage_premium_Juridical['kmeans_coverage_premium_Juridical']==16].describe()

,total_Premium,00I_Premium,01A_Premium,01J_Premium,02K_Premium,03L_Premium,04M_Premium,05E_Premium,05N_Premium,06F_Premium,07P_Premium,08H_Premium,09@_Premium,09I_Premium,10A_Premium,12L_Premium,14E_Premium,14N_Premium,15F_Premium,15O_Premium,16G_Premium,16P_Premium,18@_Premium,18I_Premium,20B_Premium,20K_Premium,25G_Premium,26H_Premium,27I_Premium,29B_Premium,29K_Premium,32N_Premium,33F_Premium,33O_Premium,34P_Premium,35H_Premium,36I_Premium,37J_Premium,40M_Premium,41E_Premium,41N_Premium,42F_Premium,45@_Premium,46A_Premium,47B_Premium,51O_Premium,55J_Premium,56B_Premium,56K_Premium,57C_Premium,57L_Premium,65K_Premium,66C_Premium,66L_Premium,67D_Premium,68E_Premium,68N_Premium,70G_Premium,70P_Premium,71H_Premium,72@_Premium,fsex_,fsex_1,fsex_2,kmeans_coverage_premium_Juridical
count,37.000000,37.0,37.0,37.0,37.0,37.000000,37.0,37.000000,37.000000,37.000000,37.0,37.000000,37.000000,37.0,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.0,37.000000,37.000000,37.0,37.000000,37.0,37.000000,37.000000,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.000000,37.000000,37.0,37.0,37.000000,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0
mean,110823.000000,0.0,0.0,0.0,0.0,196.351351,0.0,872.081081,16822.945946,109.945946,0.0,15.540541,1554.189189,0.0,144.756757,70.189189,109.729730,2707.216216,274.162162,526.351351,2970.594595,1979.702703,50.351351,0.0,54.108108,13.648649,0.0,15.918919,0.0,1091.702703,839.324324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80267.324324,124.378378,0.0,0.0,12.486486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,16.0
std,15454.866866,0.0,0.0,0.0,0.0,414.603774,0.0,560.514019,9416.510636,668.775080,0.0,94.529418,1750.455319,0.0,152.958492,112.441303,217.554035,2891.947010,533.717857,1236.742770,1171.153289,553.564804,82.693281,0.0,113.559525,12.826224,0.0,54.602289,0.0,153.407747,358.405733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12378.485503,756.564140,0.0,0.0,33.840822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,91260.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,690.000000,1108.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,714.000000,484.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59033.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,16.0
25%,97970.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9575.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2109.000000,1531.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,1027.000000,591.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73175.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,16.0
50%,108017.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1121.000000,14392.000000,0.000000,0.0,0.000000,882.000000,0.0,164.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2881.000000,2032.000000,0.000000,0.0,0.000000,21.000000,0.0,0.000000,0.0,1082.000000,642.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77051.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,16.0
75%,121344.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1251.000000,24992.000000,0.000000,0.0,0.000000,3131.000000,0.0,300.000000,162.000000,0.000000,5554.000000,0.000000,0.000000,3749.000000,2240.000000,101.000000,0.0,0.000000,26.000000,0.0,0.000000,0.0,1210.000000,976.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87147.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,16.0
max,149586.000000,0.0,0.0,0.0,0.0,1163.000000,0.0,1626.000000,35075.000000,4068.000000,0.0,575.000000,5261.000000,0.0,500.000000,372.000000,571.000000,6774.000000,1420.000000,3898.000000,5966.000000,3734.000000,202.000000,0.0,286.0000

In [200]:
coverage_premium_Nature = coverage_premium_fsex[(coverage_premium_fsex['fsex_1']==1)|(coverage_premium_fsex['fsex_2']==1)]
kmeans_fit_coverage_premium_Nature = cluster.KMeans(n_clusters = 25).fit(coverage_premium_Nature.iloc[:,0:-4]) #*****調整群數改這裡

kmeans_coverage_premium_labels = kmeans_fit_coverage_premium_Nature.labels_
print("分群結果：")
print(kmeans_coverage_premium_labels)
print("---")
print(coverage_premium_Nature.shape)
coverage_premium_Nature['kmeans_coverage_Nature'] = kmeans_coverage_premium_labels
coverage_premium_Nature['kmeans_coverage_Nature'].value_counts()

分群結果：
[23 23 23 ...  0 22  2]
---
(294958, 65)


d:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


23    114156
0      61955
4      46816
7      23491
22     15468
21      9925
16      8531
2       3257
18      2023
1       2000
9       1487
20      1320
8        987
17       782
10       672
5        489
24       488
3        322
14       266
15       216
11       125
12        86
13        77
19        13
6          6
Name: kmeans_coverage_Nature, dtype: int64

In [201]:
coverage_premium_Nature[coverage_premium_Nature['kmeans_coverage_Nature']==11].describe()

,total_Premium,00I_Premium,01A_Premium,01J_Premium,02K_Premium,03L_Premium,04M_Premium,05E_Premium,05N_Premium,06F_Premium,07P_Premium,08H_Premium,09@_Premium,09I_Premium,10A_Premium,12L_Premium,14E_Premium,14N_Premium,15F_Premium,15O_Premium,16G_Premium,16P_Premium,18@_Premium,18I_Premium,20B_Premium,20K_Premium,25G_Premium,26H_Premium,27I_Premium,29B_Premium,29K_Premium,32N_Premium,33F_Premium,33O_Premium,34P_Premium,35H_Premium,36I_Premium,37J_Premium,40M_Premium,41E_Premium,41N_Premium,42F_Premium,45@_Premium,46A_Premium,47B_Premium,51O_Premium,55J_Premium,56B_Premium,56K_Premium,57C_Premium,57L_Premium,65K_Premium,66C_Premium,66L_Premium,67D_Premium,68E_Premium,68N_Premium,70G_Premium,70P_Premium,71H_Premium,72@_Premium,fsex_,fsex_1,fsex_2,kmeans_coverage_Nature
count,125.000000,125.0,125.000000,125.0,125.000000,125.000000,125.0,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.0,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.0,125.000000,125.000000,125.0,125.00000,125.0,125.0000,125.000000,125.0,125.000000,125.0,125.0,125.000000,125.0,125.0,125.0,125.0,125.0,125.0,125.0,125.0,125.0,125.0,125.0,125.0,125.0,125.0,125.000000,125.0,125.0,125.0,125.0,125.0,125.0,125.0,125.0,125.0,125.0,125.0,125.000000,125.000000,125.0
mean,82236.872000,0.0,915.184000,0.0,58946.216000,50.968000,0.0,498.016000,12464.168000,16.000000,5.304000,9.200000,973.408000,0.0,31.136000,21.072000,93.528000,1284.344000,283.424000,292.432000,2660.400000,1980.200000,37.392000,0.0,22.472000,10.584000,0.0,15.47200,0.0,887.6160,649.768000,0.0,48.736000,0.0,0.0,30.008000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.824000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.608000,0.392000,11.0
std,13994.796297,0.0,4293.502943,0.0,13364.079586,205.033433,0.0,406.957991,7656.520039,178.885438,59.300523,72.438517,1206.176872,0.0,84.876508,79.586955,204.949308,2538.362601,863.895251,851.804142,1259.844441,878.550694,71.865678,0.0,76.636193,12.643489,0.0,55.92111,0.0,379.6643,310.146665,0.0,544.885045,0.0,0.0,335.499639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.038091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.490161,0.490161,0.0
min,66876.000000,0.0,0.000000,0.0,38307.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,451.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,11.0
25%,72887.000000,0.0,0.000000,0.0,50396.000000,0.000000,0.0,0.000000,7169.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1858.000000,1435.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,770.0000,473.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,11.0
50%,77559.000000,0.0,0.000000,0.0,55527.000000,0.000000,0.0,718.000000,11704.000000,0.000000,0.000000,0.000000,280.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2300.000000,1742.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,954.0000,591.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,11.0
75%,85000.000000,0.0,0.000000,0.0,64168.000000,0.000000,0.0,828.000000,16220.000000,0.000000,0.000000,0.000000,1887.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3182.000000,2287.000000,0.000000,0.0,0.000000,26.000000,0.0,0.00000,0.0,1082.0000,976.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,11.0
max,150063.000000,0.0,31931.000000,0.0,111278.000000,139

In [202]:
coverage_premium_Nature[coverage_premium_Nature['kmeans_coverage_Nature']==0].describe()

,total_Premium,00I_Premium,01A_Premium,01J_Premium,02K_Premium,03L_Premium,04M_Premium,05E_Premium,05N_Premium,06F_Premium,07P_Premium,08H_Premium,09@_Premium,09I_Premium,10A_Premium,12L_Premium,14E_Premium,14N_Premium,15F_Premium,15O_Premium,16G_Premium,16P_Premium,18@_Premium,18I_Premium,20B_Premium,20K_Premium,25G_Premium,26H_Premium,27I_Premium,29B_Premium,29K_Premium,32N_Premium,33F_Premium,33O_Premium,34P_Premium,35H_Premium,36I_Premium,37J_Premium,40M_Premium,41E_Premium,41N_Premium,42F_Premium,45@_Premium,46A_Premium,47B_Premium,51O_Premium,55J_Premium,56B_Premium,56K_Premium,57C_Premium,57L_Premium,65K_Premium,66C_Premium,66L_Premium,67D_Premium,68E_Premium,68N_Premium,70G_Premium,70P_Premium,71H_Premium,72@_Premium,fsex_,fsex_1,fsex_2,kmeans_coverage_Nature
count,61955.000000,61955.0,61955.000000,61955.0,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.0,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.0,61955.000000,61955.000000,61955.0,61955.000000,61955.000000,61955.0,61955.0,61955.0,61955.0,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.000000,61955.0,61955.0,61955.0,61955.000000,61955.000000,61955.0,61955.0,61955.0,61955.0,61955.0,61955.000000,61955.000000,61955.000000,61955.000000,61955.0,61955.000000,61955.000000,61955.0
mean,1650.525785,0.0,0.022742,0.0,0.254346,0.019272,13.923154,0.147575,11.436155,0.046243,0.190880,0.131644,0.410136,0.006101,0.473763,4.235849,0.814236,0.0,10.435251,1.223307,749.693358,704.479800,6.348253,0.090808,42.975870,2.819821,0.023291,30.972012,0.0,26.953821,34.879235,0.0,0.000662,0.001905,0.0,0.0,0.0,0.0,0.555968,0.341377,1.257493,0.997789,0.043144,0.036333,0.057994,3.307433,0.149996,0.0,0.0,0.0,0.452183,0.005343,0.0,0.0,0.0,0.0,0.0,0.154564,0.111048,0.025486,0.020144,0.0,0.511000,0.489000,0.0
std,685.369789,0.0,3.336768,0.0,23.397082,0.753670,166.575220,2.099018,74.612095,5.006153,9.381183,7.363316,6.873791,1.518636,9.125481,19.362636,15.024788,0.0,91.510045,29.629595,385.168331,345.061189,25.992847,6.010967,95.797303,7.416248,1.534031,75.304612,0.0,112.127480,92.023191,0.0,0.164720,0.474071,0.0,0.0,0.0,0.0,24.322405,15.231704,32.266679,26.429130,7.963910,1.678425,2.349635,84.029324,16.445211,0.0,0.0,0.0,4.740574,0.304787,0.0,0.0,0.0,0.0,0.0,14.227778,10.208695,3.933037,2.648888,0.0,0.499883,0.499883,0.0
min,9.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
25%,1405.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,576.000000,644.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
50%,1868.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,844.000000,801.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0

In [203]:
coverage_premium_Nature[coverage_premium_Nature['kmeans_coverage_Nature']==3].describe()

,total_Premium,00I_Premium,01A_Premium,01J_Premium,02K_Premium,03L_Premium,04M_Premium,05E_Premium,05N_Premium,06F_Premium,07P_Premium,08H_Premium,09@_Premium,09I_Premium,10A_Premium,12L_Premium,14E_Premium,14N_Premium,15F_Premium,15O_Premium,16G_Premium,16P_Premium,18@_Premium,18I_Premium,20B_Premium,20K_Premium,25G_Premium,26H_Premium,27I_Premium,29B_Premium,29K_Premium,32N_Premium,33F_Premium,33O_Premium,34P_Premium,35H_Premium,36I_Premium,37J_Premium,40M_Premium,41E_Premium,41N_Premium,42F_Premium,45@_Premium,46A_Premium,47B_Premium,51O_Premium,55J_Premium,56B_Premium,56K_Premium,57C_Premium,57L_Premium,65K_Premium,66C_Premium,66L_Premium,67D_Premium,68E_Premium,68N_Premium,70G_Premium,70P_Premium,71H_Premium,72@_Premium,fsex_,fsex_1,fsex_2,kmeans_coverage_Nature
count,322.000000,322.0,322.000000,322.0,322.0,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.0,322.0000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.0,322.000000,322.000000,322.0,322.000000,322.0,322.000000,322.000000,322.0,322.0,322.0,322.0,322.0,322.0,322.0,322.000000,322.000000,322.000000,322.000000,322.0,322.000000,322.0,322.0,322.000000,322.000000,322.000000,322.0,322.000000,322.0,322.0,322.0,322.0,322.0,322.0,322.0,322.0,322.0,322.0,322.0,322.000000,322.000000,322.0
mean,67847.527950,0.0,244.720497,0.0,0.0,50.462733,7.909938,526.453416,7505.531056,271.503106,11.711180,28.437888,601.559006,0.0,80.0000,28.729814,36.689441,1125.925466,150.763975,125.043478,2453.670807,1716.195652,32.313665,0.0,38.034161,13.385093,0.0,5.416149,0.0,973.801242,577.180124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.111801,8.819876,6.894410,6.124224,0.0,0.664596,0.0,0.0,51083.857143,77.956522,38.096273,0.0,6.565217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.562112,0.437888,3.0
std,7053.831404,0.0,1279.274818,0.0,0.0,180.391268,83.279965,364.801947,5421.986631,1035.241111,78.826086,121.172533,824.506973,0.0,134.2779,74.568865,132.390117,2272.487638,408.767222,588.728254,1131.028844,700.249267,64.533681,0.0,97.073255,12.544283,0.0,34.048995,0.0,325.577222,300.625317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,166.110761,111.737224,87.343887,77.586552,0.0,8.419636,0.0,0.0,6401.355880,568.620551,416.609569,0.0,24.227834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.496899,0.496899,0.0
min,54524.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,31794.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,3.0
25%,62181.500000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,3777.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,1704.000000,1266.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,887.000000,484.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,47261.750000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,3.0
50%,66667.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,721.500000,6375.000000,0.000000,0.000000,0.000000,45.500000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,2294.500000,1543.000000,0.000000,0.0,0.000000,21.000000,0.0,0.000000,0.0,1082.000000,591.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,50835.500000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,3.0
75%,72757.750000,0.0,0.000000,0.0,0.0,0.000000,0.000000,795.000000,10285.750000,0.000000,0.000000,0.000000,961.250000,0.0,200.0000,0.000000,0.000000,0.000000,0.000000,0.000000,2831.000000,2040.000000,0.000000,0.0,0.000000,26.000000,0.0,0.000000,0.0,1082.000000,591.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [204]:
coverage_premium_Nature.head(5)

,total_Premium,00I_Premium,01A_Premium,01J_Premium,02K_Premium,03L_Premium,04M_Premium,05E_Premium,05N_Premium,06F_Premium,07P_Premium,08H_Premium,09@_Premium,09I_Premium,10A_Premium,12L_Premium,14E_Premium,14N_Premium,15F_Premium,15O_Premium,16G_Premium,16P_Premium,18@_Premium,18I_Premium,20B_Premium,20K_Premium,25G_Premium,26H_Premium,27I_Premium,29B_Premium,29K_Premium,32N_Premium,33F_Premium,33O_Premium,34P_Premium,35H_Premium,36I_Premium,37J_Premium,40M_Premium,41E_Premium,41N_Premium,42F_Premium,45@_Premium,46A_Premium,47B_Premium,51O_Premium,55J_Premium,56B_Premium,56K_Premium,57C_Premium,57L_Premium,65K_Premium,66C_Premium,66L_Premium,67D_Premium,68E_Premium,68N_Premium,70G_Premium,70P_Premium,71H_Premium,72@_Premium,Policy_Number,fsex_,fsex_1,fsex_2,kmeans_coverage_Nature
0,2714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1551.0,1163.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79110176bf64b5094c19aad785aeac56e36cb609,0,0,1,23
1,3129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2031.0,1098.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61a449f6180b23379a584af904d7412205d8ff8c,0,0,1,23
2,3284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,149.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1289.0,1846.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1a1f5f582bf1ff23467352e8f2fd31acefddc654,0,0,1,23
3,183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,183.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6b420a887376f4dce53e22e600f2ca56aadec0ee,0,1,0,0
5,4941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2111.0,1096.0,993.0,634.0,0.0,107.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8a1bcfe8f801ec0b9230d5e2eafeb025c6b4945f,0,1,0,4


In [205]:
coverage_premium_Nature = coverage_premium_Nature[['Policy_Number','kmeans_coverage_Nature']]
coverage_premium_Juridical = coverage_premium_Juridical[['Policy_Number','kmeans_coverage_premium_Juridical']]

In [211]:
print(coverage_premium_Nature.shape)

(294958, 2)


In [212]:
kmeans_coverage_premium = df_policy_coverage.merge(coverage_premium_Nature, how='left', on ='Policy_Number')

In [216]:
kmeans_coverage_premium = kmeans_coverage_premium.merge(coverage_premium_Juridical, how='left', on ='Policy_Number')

In [229]:
kmeans_coverage_premium.drop(kmeans_coverage_premium.columns[1:-2],axis=1)

,Policy_Number,kmeans_coverage_Nature,kmeans_coverage_premium_Juridical
0,79110176bf64b5094c19aad785aeac56e36cb609,23.0,NaN
1,61a449f6180b23379a584af904d7412205d8ff8c,23.0,NaN
2,1a1f5f582bf1ff23467352e8f2fd31acefddc654,23.0,NaN
3,6b420a887376f4dce53e22e600f2ca56aadec0ee,0.0,NaN
4,15a5e6b7c385d23c01bf4eaa49f310d638d7d3d9,NaN,3.0
5,8a1bcfe8f801ec0b9230d5e2eafeb025c6b4945f,4.0,NaN
6,17ccb5ec55e642270298b3bbc959ff7c0ecfa6b1,23.0,NaN
7,9f4f41b3412390a9cb4b125f61aee2626317d34f,23.0,NaN
8,8004afed773c3ca2f9187cf035a89be9f65cadae,23.0,NaN
9,a6a67a68608192aa1ed5debc11d1b8a91a48716c,23.0,NaN


In [230]:
kmeans_coverage_premium = kmeans_coverage_premium.drop(kmeans_coverage_premium.columns[1:-2],axis=1)

In [232]:
kmeans_coverage_premium = pd.get_dummies(kmeans_coverage_premium, columns = ['kmeans_coverage_Nature','kmeans_coverage_premium_Juridical'])

In [233]:
kmeans_coverage_premium.head(5)

,Policy_Number,kmeans_coverage_Nature_0.0,kmeans_coverage_Nature_1.0,kmeans_coverage_Nature_2.0,kmeans_coverage_Nature_3.0,kmeans_coverage_Nature_4.0,kmeans_coverage_Nature_5.0,kmeans_coverage_Nature_6.0,kmeans_coverage_Nature_7.0,kmeans_coverage_Nature_8.0,kmeans_coverage_Nature_9.0,kmeans_coverage_Nature_10.0,kmeans_coverage_Nature_11.0,kmeans_coverage_Nature_12.0,kmeans_coverage_Nature_13.0,kmeans_coverage_Nature_14.0,kmeans_coverage_Nature_15.0,kmeans_coverage_Nature_16.0,kmeans_coverage_Nature_17.0,kmeans_coverage_Nature_18.0,kmeans_coverage_Nature_19.0,kmeans_coverage_Nature_20.0,kmeans_coverage_Nature_21.0,kmeans_coverage_Nature_22.0,kmeans_coverage_Nature_23.0,kmeans_coverage_Nature_24.0,kmeans_coverage_premium_Juridical_0.0,kmeans_coverage_premium_Juridical_1.0,kmeans_coverage_premium_Juridical_2.0,kmeans_coverage_premium_Juridical_3.0,kmeans_coverage_premium_Juridical_4.0,kmeans_coverage_premium_Juridical_5.0,kmeans_coverage_premium_Juridical_6.0,kmeans_coverage_premium_Juridical_7.0,kmeans_coverage_premium_Juridical_8.0,kmeans_coverage_premium_Juridical_9.0,kmeans_coverage_premium_Juridical_10.0,kmeans_coverage_premium_Juridical_11.0,kmeans_coverage_premium_Juridical_12.0,kmeans_coverage_premium_Juridical_13.0,kmeans_coverage_premium_Juridical_14.0,kmeans_coverage_premium_Juridical_15.0,kmeans_coverage_premium_Juridical_16.0,kmeans_coverage_premium_Juridical_17.0,kmeans_coverage_premium_Juridical_18.0,kmeans_coverage_premium_Juridical_19.0,kmeans_coverage_premium_Juridical_20.0,kmeans_coverage_premium_Juridical_21.0,kmeans_coverage_premium_Juridical_22.0,kmeans_coverage_premium_Juridical_23.0,kmeans_coverage_premium_Juridical_24.0
0,79110176bf64b5094c19aad785aeac56e36cb609,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,61a449f6180b23379a584af904d7412205d8ff8c,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1a1f5f582bf1ff23467352e8f2fd31acefddc654,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6b420a887376f4dce53e22e600f2ca56aadec0ee,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,15a5e6b7c385d23c01bf4eaa49f310d638d7d3d9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [234]:
kmeans_coverage_premium.to_csv('../Tbrain_Insurance/kmeans_coverage_premium.csv',index = False) 

In [182]:
coverage_premium_Nature = pd.get_dummies(coverage_premium_Nature, columns = ['kmeans_coverage_Nature'])

In [184]:
coverage_premium_Nature.head(5)

,Policy_Number,kmeans_coverage_Nature_0,kmeans_coverage_Nature_1,kmeans_coverage_Nature_2,kmeans_coverage_Nature_3,kmeans_coverage_Nature_4,kmeans_coverage_Nature_5,kmeans_coverage_Nature_6,kmeans_coverage_Nature_7,kmeans_coverage_Nature_8,kmeans_coverage_Nature_9,kmeans_coverage_Nature_10,kmeans_coverage_Nature_11,kmeans_coverage_Nature_12,kmeans_coverage_Nature_13,kmeans_coverage_Nature_14,kmeans_coverage_Nature_15,kmeans_coverage_Nature_16,kmeans_coverage_Nature_17,kmeans_coverage_Nature_18,kmeans_coverage_Nature_19,kmeans_coverage_Nature_20,kmeans_coverage_Nature_21,kmeans_coverage_Nature_22,kmeans_coverage_Nature_23,kmeans_coverage_Nature_24
0,79110176bf64b5094c19aad785aeac56e36cb609,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,61a449f6180b23379a584af904d7412205d8ff8c,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1a1f5f582bf1ff23467352e8f2fd31acefddc654,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6b420a887376f4dce53e22e600f2ca56aadec0ee,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,8a1bcfe8f801ec0b9230d5e2eafeb025c6b4945f,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [185]:
coverage_premium_Juridical = pd.get_dummies(coverage_premium_Juridical, columns = ['kmeans_coverage_premium_Juridical'])

In [187]:
coverage_premium_Juridical.head(5)

,Policy_Number,kmeans_coverage_premium_Juridical_0,kmeans_coverage_premium_Juridical_1,kmeans_coverage_premium_Juridical_2,kmeans_coverage_premium_Juridical_3,kmeans_coverage_premium_Juridical_4,kmeans_coverage_premium_Juridical_5,kmeans_coverage_premium_Juridical_6,kmeans_coverage_premium_Juridical_7,kmeans_coverage_premium_Juridical_8,kmeans_coverage_premium_Juridical_9,kmeans_coverage_premium_Juridical_10,kmeans_coverage_premium_Juridical_11,kmeans_coverage_premium_Juridical_12,kmeans_coverage_premium_Juridical_13,kmeans_coverage_premium_Juridical_14,kmeans_coverage_premium_Juridical_15,kmeans_coverage_premium_Juridical_16,kmeans_coverage_premium_Juridical_17,kmeans_coverage_premium_Juridical_18,kmeans_coverage_premium_Juridical_19,kmeans_coverage_premium_Juridical_20,kmeans_coverage_premium_Juridical_21,kmeans_coverage_premium_Juridical_22,kmeans_coverage_premium_Juridical_23,kmeans_coverage_premium_Juridical_24
4,15a5e6b7c385d23c01bf4eaa49f310d638d7d3d9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,88e2fb95af950a7b40eb2c8ad8fcf45626c65364,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,5431130fb4fad06388f345f6e1d3e71ec1089d32,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,aeea1cb0c1e74e25447af0cf809bb9411c2f424c,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,04922b89ac75a3ec876c3e361b74339420ba57df,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [188]:
kmeans_coverage_premium = df_policy_coverage.merge(coverage_premium_Nature, how='left', on ='Policy_Number')
kmeans_coverage_premium = kmeans_coverage_premium.merge(coverage_premium_Juridical, how='left', on ='Policy_Number')

In [193]:
kmeans_coverage_premium.head(5)

,Policy_Number,total_Premium,00I_Insured_Amount1,00I_Insured_Amount2,00I_Insured_Amount3,00I_Premium,01A_Insured_Amount1,01A_Insured_Amount2,01A_Insured_Amount3,01A_Premium,01J_Insured_Amount1,01J_Insured_Amount2,01J_Insured_Amount3,01J_Premium,02K_Insured_Amount1,02K_Insured_Amount2,02K_Insured_Amount3,02K_Premium,03L_Insured_Amount1,03L_Insured_Amount2,03L_Insured_Amount3,03L_Premium,04M_Insured_Amount1,04M_Insured_Amount2,04M_Insured_Amount3,04M_Premium,05E_Insured_Amount1,05E_Insured_Amount2,05E_Insured_Amount3,05E_Premium,05N_Insured_Amount1,05N_Insured_Amount2,05N_Insured_Amount3,05N_Premium,06F_Insured_Amount1,06F_Insured_Amount2,06F_Insured_Amount3,06F_Premium,07P_Insured_Amount1,07P_Insured_Amount2,07P_Insured_Amount3,07P_Premium,08H_Insured_Amount1,08H_Insured_Amount2,08H_Insured_Amount3,08H_Premium,09@_Insured_Amount1,09@_Insured_Amount2,09@_Insured_Amount3,09@_Premium,09I_Insured_Amount1,09I_Insured_Amount2,09I_Insured_Amount3,09I_Premium,10A_Insured_Amount1,10A_Insured_Amount2,10A_Insured_Amount3,10A_Premium,12L_Insured_Amount1,12L_Insured_Amount2,12L_Insured_Amount3,12L_Premium,14E_Insured_Amount1,14E_Insured_Amount2,14E_Insured_Amount3,14E_Premium,14N_Insured_Amount1,14N_Insured_Amount2,14N_Insured_Amount3,14N_Premium,15F_Insured_Amount1,15F_Insured_Amount2,15F_Insured_Amount3,15F_Premium,15O_Insured_Amount1,15O_Insured_Amount2,15O_Insured_Amount3,15O_Premium,16G_Insured_Amount1,16G_Insured_Amount2,16G_Insured_Amount3,16G_Premium,16P_Insured_Amount1,16P_Insured_Amount2,16P_Insured_Amount3,16P_Premium,18@_Insured_Amount1,18@_Insured_Amount2,18@_Insured_Amount3,18@_Premium,18I_Insured_Amount1,18I_Insured_Amount2,18I_Insured_Amount3,18I_Premium,20B_Insured_Amount1,20B_Insured_Amount2,20B_Insured_Amount3,20B_Premium,20K_Insured_Amount1,20K_Insured_Amount2,20K_Insured_Amount3,20K_Premium,25G_Insured_Amount1,25G_Insured_Amount2,25G_Insured_Amount3,25G_Premium,26H_Insured_Amount1,26H_Insured_Amount2,26H_Insured_Amount3,26H_Premium,27I_Insured_Amount1,27I_Insured_Amount2,27I_Insured_Amount3,27I_Premium,29B_Insured_Amount1,29B_Insured_Amount2,29B_Insured_Amount3,29B_Premium,29K_Insured_Amount1,29K_Insured_Amount2,29K_Insured_Amount3,29K_Premium,32N_Insured_Amount1,32N_Insured_Amount2,32N_Insured_Amount3,32N_Premium,33F_Insured_Amount1,33F_Insured_Amount2,33F_Insured_Amount3,33F_Premium,33O_Insured_Amount1,33O_Insured_Amount2,33O_Insured_Amount3,33O_Premium,34P_Insured_Amount1,34P_Insured_Amount2,34P_Insured_Amount3,34P_Premium,35H_Insured_Amount1,35H_Insured_Amount2,35H_Insured_Amount3,35H_Premium,36I_Insured_Amount1,36I_Insured_Amount2,36I_Insured_Amount3,36I_Premium,37J_Insured_Amount1,37J_Insured_Amount2,37J_Insured_Amount3,37J_Premium,40M_Insured_Amount1,40M_Insured_Amount2,40M_Insured_Amount3,40M_Premium,41E_Insured_Amount1,41E_Insured_Amount2,41E_Insured_Amount3,41E_Premium,41N_Insured_Amount1,41N_Insured_Amount2,41N_Insured_Amount3,41N_Premium,42F_Insured_Amount1,42F_Insured_Amount2,42F_Insured_Amount3,42F_Premium,45@_Insured_Amount1,45@_Insured_Amount2,45@_Insured_Amount3,45@_Premium,46A_Insured_Amount1,46A_Insured_Amount2,46A_Insured_Amount3,46A_Premium,47B_Insured_Amount1,47B_Insured_Amount2,47B_Insured_Amount3,47B_Premium,51O_Insured_Amount1,51O_Insured_Amount2,51O_Insured_Amount3,51O_Premium,55J_Insured_Amount1,55J_Insured_Amount2,55J_Insured_Amount3,55J_Premium,56B_Insured_Amount1,56B_Insured_Amount2,56B_Insured_Amount3,56B_Premium,56K_Insured_Amount1,56K_Insured_Amount2,56K_Insured_Amount3,56K_Premium,57C_Insured_Amount1,57C_Insured_Amount2,57C_Insured_Amount3,57C_Premium,57L_Insured_Amount1,57L_Insured_Amount2,57L_Insured_Amount3,57L_Premium,65K_Insured_Amount1,65K_Insured_Amount2,65K_Insured_Amount3,65K_Premium,66C_Insured_Amount1,66C_Insured_Amount2,66C_Insured_Amount3,66C_Premium,66L_Insured_Amount1,66L_Insured_Amount2,66L_Insured_Amount3,66L_Premium,67D_Insured_Amount1,67D_Insured_Amount2,67D_Insured_Amount3,67D_Premium,68E_Insured_Amount1,68E_Insured_Amount2,68E_Insured_Amount3,68E_Premium,68N_Insured

In [191]:
df_policy_coverage.isnull().values.any()

False

In [195]:
kmeans_coverage_premium.iloc[:,61:-1] = kmeans_coverage_premium.iloc[:,61:-1].fillna(value=0)

MemoryError: 

In [ ]:
# # Nature
# df1 = df_all[(df_all['fsex_1']==1)|(df_all['fsex_2']==1)]

# df_train1 = df1[df1['Next_Premium']!=100]
# print(df_train1.shape)
# df_test1 = df1[df1['Next_Premium']==100]
# print(df_test1.shape)
# print(df_train1.shape[0]+df_test1.shape[0])
# print(df1.shape[0])

# # Juridical
# df2 = df_all[(df_all['fsex_ ']==1)]
# df2 = df2.drop(['i_age', 'd_age'], axis=1)    ######

# df_train2 = df2[df2['Next_Premium']!=100]
# print(df_train2.shape)
# df_test2 = df2[df2['Next_Premium']==100]
# print(df_test2.shape)
# print(df_train2.shape[0]+df_test2.shape[0])
# print(df2.shape[0])

In [7]:
kmeans_fit_coverage_premium = cluster.KMeans(n_clusters = 5).fit(coverage_premium)

In [8]:
policy_premium_cluster_labels = kmeans_fit_coverage_premium.labels_
print("分群結果：")
print(policy_premium_cluster_labels)
print("---")

分群結果：
[0 0 0 ... 0 2 2]
---


In [9]:
df_policy_coverage['kmeans_cluster_by_coverage_premium'] = policy_premium_cluster_labels

In [10]:
df_policy_coverage['kmeans_cluster_by_coverage_premium'].value_counts()

0    300355
2     38863
1      8233
3      3006
4       816
Name: kmeans_cluster_by_coverage_premium, dtype: int64

In [11]:
df_policy_coverage.loc[df_policy_coverage['kmeans_cluster_by_coverage_premium']==0].head(10)

,Policy_Number,total_Premium,00I_Insured_Amount1,00I_Insured_Amount2,00I_Insured_Amount3,00I_Premium,01A_Insured_Amount1,01A_Insured_Amount2,01A_Insured_Amount3,01A_Premium,01J_Insured_Amount1,01J_Insured_Amount2,01J_Insured_Amount3,01J_Premium,02K_Insured_Amount1,02K_Insured_Amount2,02K_Insured_Amount3,02K_Premium,03L_Insured_Amount1,03L_Insured_Amount2,03L_Insured_Amount3,03L_Premium,04M_Insured_Amount1,04M_Insured_Amount2,04M_Insured_Amount3,04M_Premium,05E_Insured_Amount1,05E_Insured_Amount2,05E_Insured_Amount3,05E_Premium,05N_Insured_Amount1,05N_Insured_Amount2,05N_Insured_Amount3,05N_Premium,06F_Insured_Amount1,06F_Insured_Amount2,06F_Insured_Amount3,06F_Premium,07P_Insured_Amount1,07P_Insured_Amount2,07P_Insured_Amount3,07P_Premium,08H_Insured_Amount1,08H_Insured_Amount2,08H_Insured_Amount3,08H_Premium,09@_Insured_Amount1,09@_Insured_Amount2,09@_Insured_Amount3,09@_Premium,09I_Insured_Amount1,09I_Insured_Amount2,09I_Insured_Amount3,09I_Premium,10A_Insured_Amount1,10A_Insured_Amount2,10A_Insured_Amount3,10A_Premium,12L_Insured_Amount1,12L_Insured_Amount2,12L_Insured_Amount3,12L_Premium,14E_Insured_Amount1,14E_Insured_Amount2,14E_Insured_Amount3,14E_Premium,14N_Insured_Amount1,14N_Insured_Amount2,14N_Insured_Amount3,14N_Premium,15F_Insured_Amount1,15F_Insured_Amount2,15F_Insured_Amount3,15F_Premium,15O_Insured_Amount1,15O_Insured_Amount2,15O_Insured_Amount3,15O_Premium,16G_Insured_Amount1,16G_Insured_Amount2,16G_Insured_Amount3,16G_Premium,16P_Insured_Amount1,16P_Insured_Amount2,16P_Insured_Amount3,16P_Premium,18@_Insured_Amount1,18@_Insured_Amount2,18@_Insured_Amount3,18@_Premium,18I_Insured_Amount1,18I_Insured_Amount2,18I_Insured_Amount3,18I_Premium,20B_Insured_Amount1,20B_Insured_Amount2,20B_Insured_Amount3,20B_Premium,20K_Insured_Amount1,20K_Insured_Amount2,20K_Insured_Amount3,20K_Premium,25G_Insured_Amount1,25G_Insured_Amount2,25G_Insured_Amount3,25G_Premium,26H_Insured_Amount1,26H_Insured_Amount2,26H_Insured_Amount3,26H_Premium,27I_Insured_Amount1,27I_Insured_Amount2,27I_Insured_Amount3,27I_Premium,29B_Insured_Amount1,29B_Insured_Amount2,29B_Insured_Amount3,29B_Premium,29K_Insured_Amount1,29K_Insured_Amount2,29K_Insured_Amount3,29K_Premium,32N_Insured_Amount1,32N_Insured_Amount2,32N_Insured_Amount3,32N_Premium,33F_Insured_Amount1,33F_Insured_Amount2,33F_Insured_Amount3,33F_Premium,33O_Insured_Amount1,33O_Insured_Amount2,33O_Insured_Amount3,33O_Premium,34P_Insured_Amount1,34P_Insured_Amount2,34P_Insured_Amount3,34P_Premium,35H_Insured_Amount1,35H_Insured_Amount2,35H_Insured_Amount3,35H_Premium,36I_Insured_Amount1,36I_Insured_Amount2,36I_Insured_Amount3,36I_Premium,37J_Insured_Amount1,37J_Insured_Amount2,37J_Insured_Amount3,37J_Premium,40M_Insured_Amount1,40M_Insured_Amount2,40M_Insured_Amount3,40M_Premium,41E_Insured_Amount1,41E_Insured_Amount2,41E_Insured_Amount3,41E_Premium,41N_Insured_Amount1,41N_Insured_Amount2,41N_Insured_Amount3,41N_Premium,42F_Insured_Amount1,42F_Insured_Amount2,42F_Insured_Amount3,42F_Premium,45@_Insured_Amount1,45@_Insured_Amount2,45@_Insured_Amount3,45@_Premium,46A_Insured_Amount1,46A_Insured_Amount2,46A_Insured_Amount3,46A_Premium,47B_Insured_Amount1,47B_Insured_Amount2,47B_Insured_Amount3,47B_Premium,51O_Insured_Amount1,51O_Insured_Amount2,51O_Insured_Amount3,51O_Premium,55J_Insured_Amount1,55J_Insured_Amount2,55J_Insured_Amount3,55J_Premium,56B_Insured_Amount1,56B_Insured_Amount2,56B_Insured_Amount3,56B_Premium,56K_Insured_Amount1,56K_Insured_Amount2,56K_Insured_Amount3,56K_Premium,57C_Insured_Amount1,57C_Insured_Amount2,57C_Insured_Amount3,57C_Premium,57L_Insured_Amount1,57L_Insured_Amount2,57L_Insured_Amount3,57L_Premium,65K_Insured_Amount1,65K_Insured_Amount2,65K_Insured_Amount3,65K_Premium,66C_Insured_Amount1,66C_Insured_Amount2,66C_Insured_Amount3,66C_Premium,66L_Insured_Amount1,66L_Insured_Amount2,66L_Insured_Amount3,66L_Premium,67D_Insured_Amount1,67D_Insured_Amount2,67D_Insured_Amount3,67D_Premium,68E_Insured_Amount1,68E_Insured_Amount2,68E_Insured_Amount3,68E_Premium,68N_Insured

In [19]:
df_policy_coverage.loc[df_policy_coverage['kmeans_cluster_by_coverage_premium']==2].head(10)

,Policy_Number,total_Premium,00I_Insured_Amount1,00I_Insured_Amount2,00I_Insured_Amount3,00I_Premium,01A_Insured_Amount1,01A_Insured_Amount2,01A_Insured_Amount3,01A_Premium,01J_Insured_Amount1,01J_Insured_Amount2,01J_Insured_Amount3,01J_Premium,02K_Insured_Amount1,02K_Insured_Amount2,02K_Insured_Amount3,02K_Premium,03L_Insured_Amount1,03L_Insured_Amount2,03L_Insured_Amount3,03L_Premium,04M_Insured_Amount1,04M_Insured_Amount2,04M_Insured_Amount3,04M_Premium,05E_Insured_Amount1,05E_Insured_Amount2,05E_Insured_Amount3,05E_Premium,05N_Insured_Amount1,05N_Insured_Amount2,05N_Insured_Amount3,05N_Premium,06F_Insured_Amount1,06F_Insured_Amount2,06F_Insured_Amount3,06F_Premium,07P_Insured_Amount1,07P_Insured_Amount2,07P_Insured_Amount3,07P_Premium,08H_Insured_Amount1,08H_Insured_Amount2,08H_Insured_Amount3,08H_Premium,09@_Insured_Amount1,09@_Insured_Amount2,09@_Insured_Amount3,09@_Premium,09I_Insured_Amount1,09I_Insured_Amount2,09I_Insured_Amount3,09I_Premium,10A_Insured_Amount1,10A_Insured_Amount2,10A_Insured_Amount3,10A_Premium,12L_Insured_Amount1,12L_Insured_Amount2,12L_Insured_Amount3,12L_Premium,14E_Insured_Amount1,14E_Insured_Amount2,14E_Insured_Amount3,14E_Premium,14N_Insured_Amount1,14N_Insured_Amount2,14N_Insured_Amount3,14N_Premium,15F_Insured_Amount1,15F_Insured_Amount2,15F_Insured_Amount3,15F_Premium,15O_Insured_Amount1,15O_Insured_Amount2,15O_Insured_Amount3,15O_Premium,16G_Insured_Amount1,16G_Insured_Amount2,16G_Insured_Amount3,16G_Premium,16P_Insured_Amount1,16P_Insured_Amount2,16P_Insured_Amount3,16P_Premium,18@_Insured_Amount1,18@_Insured_Amount2,18@_Insured_Amount3,18@_Premium,18I_Insured_Amount1,18I_Insured_Amount2,18I_Insured_Amount3,18I_Premium,20B_Insured_Amount1,20B_Insured_Amount2,20B_Insured_Amount3,20B_Premium,20K_Insured_Amount1,20K_Insured_Amount2,20K_Insured_Amount3,20K_Premium,25G_Insured_Amount1,25G_Insured_Amount2,25G_Insured_Amount3,25G_Premium,26H_Insured_Amount1,26H_Insured_Amount2,26H_Insured_Amount3,26H_Premium,27I_Insured_Amount1,27I_Insured_Amount2,27I_Insured_Amount3,27I_Premium,29B_Insured_Amount1,29B_Insured_Amount2,29B_Insured_Amount3,29B_Premium,29K_Insured_Amount1,29K_Insured_Amount2,29K_Insured_Amount3,29K_Premium,32N_Insured_Amount1,32N_Insured_Amount2,32N_Insured_Amount3,32N_Premium,33F_Insured_Amount1,33F_Insured_Amount2,33F_Insured_Amount3,33F_Premium,33O_Insured_Amount1,33O_Insured_Amount2,33O_Insured_Amount3,33O_Premium,34P_Insured_Amount1,34P_Insured_Amount2,34P_Insured_Amount3,34P_Premium,35H_Insured_Amount1,35H_Insured_Amount2,35H_Insured_Amount3,35H_Premium,36I_Insured_Amount1,36I_Insured_Amount2,36I_Insured_Amount3,36I_Premium,37J_Insured_Amount1,37J_Insured_Amount2,37J_Insured_Amount3,37J_Premium,40M_Insured_Amount1,40M_Insured_Amount2,40M_Insured_Amount3,40M_Premium,41E_Insured_Amount1,41E_Insured_Amount2,41E_Insured_Amount3,41E_Premium,41N_Insured_Amount1,41N_Insured_Amount2,41N_Insured_Amount3,41N_Premium,42F_Insured_Amount1,42F_Insured_Amount2,42F_Insured_Amount3,42F_Premium,45@_Insured_Amount1,45@_Insured_Amount2,45@_Insured_Amount3,45@_Premium,46A_Insured_Amount1,46A_Insured_Amount2,46A_Insured_Amount3,46A_Premium,47B_Insured_Amount1,47B_Insured_Amount2,47B_Insured_Amount3,47B_Premium,51O_Insured_Amount1,51O_Insured_Amount2,51O_Insured_Amount3,51O_Premium,55J_Insured_Amount1,55J_Insured_Amount2,55J_Insured_Amount3,55J_Premium,56B_Insured_Amount1,56B_Insured_Amount2,56B_Insured_Amount3,56B_Premium,56K_Insured_Amount1,56K_Insured_Amount2,56K_Insured_Amount3,56K_Premium,57C_Insured_Amount1,57C_Insured_Amount2,57C_Insured_Amount3,57C_Premium,57L_Insured_Amount1,57L_Insured_Amount2,57L_Insured_Amount3,57L_Premium,65K_Insured_Amount1,65K_Insured_Amount2,65K_Insured_Amount3,65K_Premium,66C_Insured_Amount1,66C_Insured_Amount2,66C_Insured_Amount3,66C_Premium,66L_Insured_Amount1,66L_Insured_Amount2,66L_Insured_Amount3,66L_Premium,67D_Insured_Amount1,67D_Insured_Amount2,67D_Insured_Amount3,67D_Premium,68E_Insured_Amount1,68E_Insured_Amount2,68E_Insured_Amount3,68E_Premium,68N_Insured

In [18]:
df_policy_coverage.loc[df_policy_coverage['kmeans_cluster_by_coverage_premium']==1].head(10)

,Policy_Number,total_Premium,00I_Insured_Amount1,00I_Insured_Amount2,00I_Insured_Amount3,00I_Premium,01A_Insured_Amount1,01A_Insured_Amount2,01A_Insured_Amount3,01A_Premium,01J_Insured_Amount1,01J_Insured_Amount2,01J_Insured_Amount3,01J_Premium,02K_Insured_Amount1,02K_Insured_Amount2,02K_Insured_Amount3,02K_Premium,03L_Insured_Amount1,03L_Insured_Amount2,03L_Insured_Amount3,03L_Premium,04M_Insured_Amount1,04M_Insured_Amount2,04M_Insured_Amount3,04M_Premium,05E_Insured_Amount1,05E_Insured_Amount2,05E_Insured_Amount3,05E_Premium,05N_Insured_Amount1,05N_Insured_Amount2,05N_Insured_Amount3,05N_Premium,06F_Insured_Amount1,06F_Insured_Amount2,06F_Insured_Amount3,06F_Premium,07P_Insured_Amount1,07P_Insured_Amount2,07P_Insured_Amount3,07P_Premium,08H_Insured_Amount1,08H_Insured_Amount2,08H_Insured_Amount3,08H_Premium,09@_Insured_Amount1,09@_Insured_Amount2,09@_Insured_Amount3,09@_Premium,09I_Insured_Amount1,09I_Insured_Amount2,09I_Insured_Amount3,09I_Premium,10A_Insured_Amount1,10A_Insured_Amount2,10A_Insured_Amount3,10A_Premium,12L_Insured_Amount1,12L_Insured_Amount2,12L_Insured_Amount3,12L_Premium,14E_Insured_Amount1,14E_Insured_Amount2,14E_Insured_Amount3,14E_Premium,14N_Insured_Amount1,14N_Insured_Amount2,14N_Insured_Amount3,14N_Premium,15F_Insured_Amount1,15F_Insured_Amount2,15F_Insured_Amount3,15F_Premium,15O_Insured_Amount1,15O_Insured_Amount2,15O_Insured_Amount3,15O_Premium,16G_Insured_Amount1,16G_Insured_Amount2,16G_Insured_Amount3,16G_Premium,16P_Insured_Amount1,16P_Insured_Amount2,16P_Insured_Amount3,16P_Premium,18@_Insured_Amount1,18@_Insured_Amount2,18@_Insured_Amount3,18@_Premium,18I_Insured_Amount1,18I_Insured_Amount2,18I_Insured_Amount3,18I_Premium,20B_Insured_Amount1,20B_Insured_Amount2,20B_Insured_Amount3,20B_Premium,20K_Insured_Amount1,20K_Insured_Amount2,20K_Insured_Amount3,20K_Premium,25G_Insured_Amount1,25G_Insured_Amount2,25G_Insured_Amount3,25G_Premium,26H_Insured_Amount1,26H_Insured_Amount2,26H_Insured_Amount3,26H_Premium,27I_Insured_Amount1,27I_Insured_Amount2,27I_Insured_Amount3,27I_Premium,29B_Insured_Amount1,29B_Insured_Amount2,29B_Insured_Amount3,29B_Premium,29K_Insured_Amount1,29K_Insured_Amount2,29K_Insured_Amount3,29K_Premium,32N_Insured_Amount1,32N_Insured_Amount2,32N_Insured_Amount3,32N_Premium,33F_Insured_Amount1,33F_Insured_Amount2,33F_Insured_Amount3,33F_Premium,33O_Insured_Amount1,33O_Insured_Amount2,33O_Insured_Amount3,33O_Premium,34P_Insured_Amount1,34P_Insured_Amount2,34P_Insured_Amount3,34P_Premium,35H_Insured_Amount1,35H_Insured_Amount2,35H_Insured_Amount3,35H_Premium,36I_Insured_Amount1,36I_Insured_Amount2,36I_Insured_Amount3,36I_Premium,37J_Insured_Amount1,37J_Insured_Amount2,37J_Insured_Amount3,37J_Premium,40M_Insured_Amount1,40M_Insured_Amount2,40M_Insured_Amount3,40M_Premium,41E_Insured_Amount1,41E_Insured_Amount2,41E_Insured_Amount3,41E_Premium,41N_Insured_Amount1,41N_Insured_Amount2,41N_Insured_Amount3,41N_Premium,42F_Insured_Amount1,42F_Insured_Amount2,42F_Insured_Amount3,42F_Premium,45@_Insured_Amount1,45@_Insured_Amount2,45@_Insured_Amount3,45@_Premium,46A_Insured_Amount1,46A_Insured_Amount2,46A_Insured_Amount3,46A_Premium,47B_Insured_Amount1,47B_Insured_Amount2,47B_Insured_Amount3,47B_Premium,51O_Insured_Amount1,51O_Insured_Amount2,51O_Insured_Amount3,51O_Premium,55J_Insured_Amount1,55J_Insured_Amount2,55J_Insured_Amount3,55J_Premium,56B_Insured_Amount1,56B_Insured_Amount2,56B_Insured_Amount3,56B_Premium,56K_Insured_Amount1,56K_Insured_Amount2,56K_Insured_Amount3,56K_Premium,57C_Insured_Amount1,57C_Insured_Amount2,57C_Insured_Amount3,57C_Premium,57L_Insured_Amount1,57L_Insured_Amount2,57L_Insured_Amount3,57L_Premium,65K_Insured_Amount1,65K_Insured_Amount2,65K_Insured_Amount3,65K_Premium,66C_Insured_Amount1,66C_Insured_Amount2,66C_Insured_Amount3,66C_Premium,66L_Insured_Amount1,66L_Insured_Amount2,66L_Insured_Amount3,66L_Premium,67D_Insured_Amount1,67D_Insured_Amount2,67D_Insured_Amount3,67D_Premium,68E_Insured_Amount1,68E_Insured_Amount2,68E_Insured_Amount3,68E_Premium,68N_Insured

In [21]:
df_policy.loc[df_policy['Policy_Number'].isin(df_policy_coverage.loc[df_policy_coverage['kmeans_cluster_by_coverage_premium']==2].head(10)['Policy_Number'])]

,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,fpt,Main_Insurance_Coverage_Group,Insurance_Coverage,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,fassured,ibirth,fsex,fmarriage,aassured_zip,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9
52,e8ce526c61030175b6191f82554bd1847e7ae0e6,Fa6db635dd548a8a55f9a3931dc19978b0db9f773,4e5367a93aeb2e82d0f62134e0003625350ea872,,0cb06d660d668620fed4ee67511f1a0360073cfc,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2011,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,車損,04M,0,0,1474000,0,13965,288.0,4ed9c4a317c5b312ba1d8f263a06338c7f80fc6e,0.0,-1,-0.38,0.0,1,02/1960,1,2,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,02/1960,0,0,0,0,0,0,0,
53,e8ce526c61030175b6191f82554bd1847e7ae0e6,Fa6db635dd548a8a55f9a3931dc19978b0db9f773,4e5367a93aeb2e82d0f62134e0003625350ea872,,0cb06d660d668620fed4ee67511f1a0360073cfc,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2011,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,竊盜,05N,0,0,1474000,10,2769,288.0,4ed9c4a317c5b312ba1d8f263a06338c7f80fc6e,0.0,-1,-0.38,0.0,1,02/1960,1,2,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,02/1960,0,0,0,0,0,0,0,
54,e8ce526c61030175b6191f82554bd1847e7ae0e6,Fa6db635dd548a8a55f9a3931dc19978b0db9f773,4e5367a93aeb2e82d0f62134e0003625350ea872,,0cb06d660d668620fed4ee67511f1a0360073cfc,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2011,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,車責,16G,3000000,0,6000000,0,1280,288.0,4ed9c4a317c5b312ba1d8f263a06338c7f80fc6e,0.0,-1,-0.38,0.0,1,02/1960,1,2,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,02/1960,0,0,0,0,0,0,0,
55,e8ce526c61030175b6191f82554bd1847e7ae0e6,Fa6db635dd548a8a55f9a3931dc19978b0db9f773,4e5367a93aeb2e82d0f62134e0003625350ea872,,0cb06d660d668620fed4ee67511f1a0360073cfc,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2011,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,車責,16P,0,0,500000,0,973,288.0,4ed9c4a317c5b312ba1d8f263a06338c7f80fc6e,0.0,-1,-0.38,0.0,1,02/1960,1,2,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,02/1960,0,0,0,0,0,0,0,
56,e8ce526c61030175b6191f82554bd1847e7ae0e6,Fa6db635dd548a8a55f9a3931dc19978b0db9f773,4e5367a93aeb2e82d0f62134e0003625350ea872,,0cb06d660d668620fed4ee67511f1a0360073cfc,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2011,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,車責,29B,0,3000000,12000000,0,887,288.0,4ed9c4a317c5b312ba1d8f263a06338c7f80fc6e,0.0,-1,-0.38,0.0,1,02/1960,1,2,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,02/1960,0,0,0,0,0,0,0,
57,e8ce526c61030175b6191f82554bd1847e7ae0e6,Fa6db635dd548a8a55f9a3931dc19978b0db9f773,4e5367a93aeb2e82d0f62134e0003625350ea872,,0cb06d660d668620fed4ee67511f1a0360073cfc,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2011,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,車責,29K,0,0,3000000,0,479,288.0,4ed9c4a317c5b312ba1d8f263a06338c7f80fc6e,0.0,-1,-0.38,0.0,1,02/1960,1,2,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,02/1960,0,0,0,0,0,0,0,
107,9243d3518aa0cce1eaf6c7f79b7711078a739bfd,2d9d39a8b4b8555859c3b6f2da7b33996f97e1360,NaN,Y,748696cea31f54eaa880eb5c1c654d1e20efcfaf,9e1816a8c2a63b7f96a82ce6487b2d43222750db,a84397beb9b

In [16]:
df_policy['fassured'].value_counts()

1    1427579
2     317771
3       2586
6          6
Name: fassured, dtype: int64